In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from keras.models import load_model
from recognize_numbers import recognize


C:\Users\mivog\AppData\Local\Programs\Python\Python36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_data(training_data_directory):
    print("Loading data")
    file_name = os.path.join(training_data_directory, "test_data-1.npy")
    training_data_ = np.load(file_name)
    print("Data with length", len(training_data_), "loaded")
    return training_data_


training_data = load_data(r"E:\Trackmania Data\training_data_new")


Loading data


Data with length 2185 loaded


In [3]:
velocity_images = training_data[:, 1]
img_digits = [Image.new("RGB", (18, 32)) for _ in range(3 * len(velocity_images))] 
for i in range(len(velocity_images)):
    img_digits[3 * i].paste(Image.fromarray(velocity_images[i][:, :18]))
    img_digits[3 * i + 1].paste(Image.fromarray(velocity_images[i][:, 17:35]))
    img_digits[3 * i + 2].paste(Image.fromarray(velocity_images[i][:, 35:53]))
img_digits = [np.asarray(e) for e in img_digits]

In [4]:
velo_recognition_model = load_model(r"C:\Users\mivog\PycharmProjects\driving ai trackmania\velocity_recognition-100.h5")


In [5]:
digits_with_confidence_value = recognize(np.array(img_digits)/255, velo_recognition_model)

In [6]:
confidences = [e[1] for e in digits_with_confidence_value] 
confidences[2:6]

[0.9999838, 0.95069075, 1.0, 0.9999838]

In [7]:
min(confidences)

0.9441895

In [8]:
velocities = [int('{}{}{}'.format(digits_with_confidence_value[3 * i][0], digits_with_confidence_value[3 * i + 1][0],
                                  digits_with_confidence_value[3 * i + 2][0]))
              for i in range(len(velocity_images))]


In [9]:
velocities = velocities[22:]
training_data = training_data[22:]
assert len(velocities) == len(training_data)

In [10]:
key_presses = training_data[:, 2]

In [11]:
def one_hot(keys):
    key_dict = {
        "W": 0,
        "S": 1,
        "A": 2,
        "D": 3
    }
    ar = [0] * 4
    for key in keys:
        ar[key_dict[key]] = 1
    return ar

In [12]:
key_presses_one_hot = [one_hot(e) for e in key_presses]

In [13]:
da = pd.DataFrame({'Velocities': pd.Series(velocities), "W": pd.Series([e[0] for e in key_presses_one_hot]),
                   "S": pd.Series([e[1] for e in key_presses_one_hot]),
                   "A": pd.Series([e[2] for e in key_presses_one_hot]),
                   "D": pd.Series([e[3] for e in key_presses_one_hot])})


In [14]:
da.describe()

,A,D,S,Velocities,W
count,2163.000000,2163.000000,2163.0,2163.000000,2163.000000
mean,0.248266,0.039760,0.0,321.095700,0.978733
std,0.432107,0.195439,0.0,102.122456,0.144306
min,0.000000,0.000000,0.0,77.000000,0.000000
25%,0.000000,0.000000,0.0,249.000000,1.000000
50%,0.000000,0.000000,0.0,348.000000,1.000000
75%,0.000000,0.000000,0.0,401.000000,1.000000
max,1.000000,1.000000,0.0,484.000000,1.000000


In [15]:
da.corr()

,A,D,S,Velocities,W
A,1.000000,-0.116939,NaN,0.192482,-0.211996
D,-0.116939,1.000000,NaN,-0.115067,0.029995
S,NaN,NaN,NaN,NaN,NaN
Velocities,0.192482,-0.115067,NaN,1.000000,-0.057487
W,-0.211996,0.029995,NaN,-0.057487,1.000000


In [16]:
plt.imshow(training_data[0, 0])
plt.show()


In [17]:
plt.plot(velocities)
plt.show()

In [18]:
velocities_diff = np.diff(velocities)


In [19]:
plt.plot(velocities_diff)
# plt.ylim([-5, 5])
plt.show()

In [20]:
np.mean(velocities_diff)

0.13506012950971322

In [32]:
# binned, bin_edges = np.histogram(velocities, bins="auto") 
n, bin_edges, _ = plt.hist(velocities, 'auto')
# plt.show()
number_bins = len(bin_edges) - 1
print("number of bins:", number_bins)
print(bin_edges)
plt.show()

number of bins: 18
[ 77.          99.61111111 122.22222222 144.83333333 167.44444444
 190.05555556 212.66666667 235.27777778 257.88888889 280.5
 303.11111111 325.72222222 348.33333333 370.94444444 393.55555556
 416.16666667 438.77777778 461.38888889 484.        ]


In [22]:
bin_ind = np.digitize(velocities, bin_edges) - 1
for k, ind in enumerate(bin_ind):
    if ind == number_bins:
        bin_ind[k] -= 1 
velocities_binned_one_hot = []
for ind in bin_ind:
    one_hot_vector = np.zeros(number_bins)
    one_hot_vector[ind] = 1 
    velocities_binned_one_hot.append(one_hot_vector)

In [23]:
velocities.index(max(velocities))

1879

In [24]:
velocities_binned_one_hot[824]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0.])

In [25]:
da = pd.DataFrame({'Velocities': pd.Series(velocities), 
                   "W": pd.Series([e[0] for e in key_presses_one_hot]),
                   "S": pd.Series([e[1] for e in key_presses_one_hot]),
                   "A": pd.Series([e[2] for e in key_presses_one_hot]),
                   "D": pd.Series([e[3] for e in key_presses_one_hot]),
                   "Velocities_binned": pd.Series(np.digitize(velocities, bin_edges))})


In [26]:
da.corr()

,A,D,S,Velocities,Velocities_binned,W
A,1.000000,-0.116939,NaN,0.192482,0.200229,-0.211996
D,-0.116939,1.000000,NaN,-0.115067,-0.117138,0.029995
S,NaN,NaN,NaN,NaN,NaN,NaN
Velocities,0.192482,-0.115067,NaN,1.000000,0.998010,-0.057487
Velocities_binned,0.200229,-0.117138,NaN,0.998010,1.000000,-0.055237
W,-0.211996,0.029995,NaN,-0.057487,-0.055237,1.000000


In [27]:
screenshots = np.array(training_data[:, 0]) / 255

In [28]:
# SCRAPPED FOR NOW, ONLY 1 VELOCITY, BECAUSE VELOS ONLY CHANGE OVER LONGER TIMESCALE
# Model: current image, 5 latest binned velocities

# h = [1]
# for k in range(number_bins - 1):
#     h.append(0)
# helper = [np.array(h) for _ in range(5)]
# latest_five_binned_velocities = []
# for k in range(4):
#     _ = helper.pop(0)
#     helper.append(velocities_binned_one_hot[k])
#     latest_five_binned_velocities.append(helper.copy())
# 
# for k in range(4, len(training_data)):
#     latest_five_binned_velocities.append(velocities_binned_one_hot[k-4:k+1])

In [29]:
# latest_five_binned_velocities[155]

In [30]:
# velocities_binned_one_hot[100:200:25]

In [31]:
np.save(r"E:\Trackmania Data\training_data_new\binned_velo_TEST.npy", [screenshots, velocities_binned_one_hot, key_presses_one_hot])